In [2]:
from pandas_datareader import data
import datetime, datedelta
from bokeh.plotting import figure, show, output_file 
from bokeh.models.annotations import Title
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource, HoverTool

In [3]:
now = datetime.datetime.now()
start = now - datedelta.datedelta(months=3)
end = now

stock_symbol = 'AAPL'

df = None
try:
    df = data.DataReader(name=stock_symbol, data_source='yahoo', start=start, end=end)
except:
    print('Unable to fetch data')

#df.index[df.Open > df.Close]

def inc_dec(c, o):
    if c > o:
        return 'Increase'
    elif c < o:
        return 'Decrease'
    else:
        return 'Equal'
    
df['Status'] = [inc_dec(c1, o1) for c1, o1 in zip(df.Close, df.Open)]

df.index[df.Status == 'Decrease']
df['Height'] = abs(df.Open - df.Close)
df['Middle'] = (df.Open + df.Close)/2
df["Date_string"] = df.index.strftime("%Y-%m-%d")

In [4]:
p = figure(x_axis_type='datetime', width=1000, height=300, sizing_mode='scale_both')
t = Title()
t.text = 'Candlestick chart for ' + stock_symbol + ': ' + start.strftime('%d/%m/%y') + ' - ' + end.strftime('%d/%m/%y')
p.title = t
p.grid.grid_line_alpha = 0.3

hours_12 = 12*60*60*1000

#days_earn = df.index[df.Close > df.Open]
#days_lose = df.index[df.Close < df.Open]
#p.rect(days_earn, (df.Open + df.Close)/2, hours_12, abs(df.Open - df.Close), fill_color='green', line_color='black')
#p.rect(days_lose, (df.Open + df.Close)/2, hours_12, abs(df.Open - df.Close), fill_color='red', line_color='black')

hover = HoverTool(tooltips=[("Date", "@Date_string"), ("Open", "@Open"), ("Close", "@Close"), ("High", "@High"), ("Low", "@Low")])
p.add_tools(hover)

cds_all = ColumnDataSource(df)
cds_increase = ColumnDataSource(df[df['Status']=='Increase'])
cds_decrease = ColumnDataSource(df[df['Status']=='Decrease'])

#p.segment(df.index, df.High, df.index, df.Low)
p.segment('Date', 'High', 'Date', 'Low', source=cds_all)
p.rect('Date', 'Middle', hours_12, 'Height', fill_color='green', line_color='black', source=cds_increase)
p.rect('Date', 'Middle', hours_12, 'Height', fill_color='red', line_color='black', source=cds_decrease)
#p.rect(df.index[df.Status=='Increase'], df.Middle[df.Status=='Increase'], hours_12, df.Height[df.Status=='Increase'], fill_color='green', line_color='black')
#p.rect(df.index[df.Status=='Decrease'], df.Middle[df.Status=='Decrease'], hours_12, df.Height[df.Status=='Decrease'], fill_color='red', line_color='black')

script1, div1 = components(p)
cdn_js = CDN.js_files
cdn_css = CDN.css_files

output_file("CS.html")
show(p)

#df.index
#ColumnDataSource(df).data['Date']
